# Genetic Algorithm

I will explore here the concept of genetic algorithm. The idea is very simple, we design an algorithm that can reproduce a genetic evolution (survival of the fittest) which would evolve into a dna able to reproduce the expected behaviour. There are several parts of a genetic algorithm that are compulsory.

- Genetic algorithm
  - DNA structure
  - Population and Peasant
  - Fitness function
  - Crossover
  - Mutation

## DNA structure

It is as straightforwards as it looks like. The DNA is the information regarding the peasant in the population that will rule its behaviour. It can be as complex as determining the number of nodes or layers in a Neural Network or as simple as beeing a number between 0 and 1. No matter the complexity or the meaning of the DNA it has to fullfill three requirements to be considered a Genetic Algorithm DNA

- Interchangable
- Mixable
- Mutable

It is clear. I must be able to change two DNAs from peasants and the algorithm must work. I must be able to mix two DNAs to form at least one new DNA. And I must be able to mutate a single gen of the DNA and the previous two rules must not be affected.

For example, it I have a DNA that describes the sequence of 0s and 1s I should be able to fullfill the three rules.

DNA_1 = "0100110"   -> Sum : 3

DNA_2 = "1111000"   -> Sum : 4

### Interchangeable

DNA_1 = "1111000"   -> Sum : 4

DNA_2 = "0100110"   -> Sum : 3

### Mixable

DNA_3 = "1111110"   -> Sum : 6

### Mutable

DNA_3 = "1011110"   -> Sum : 5

It is a very easy example, but enough to understand what is a DNA.

## Population and Peasant

The population is the collection of Peasants. This class should have all the methods that involve the collection of Peasants such as generation of a population, crossover ... as well as record information of the current population such as fitness values, mean error value, number of Peasants above a threshold... This class is very flexible and sometimes it can be ommited. 

Personally, I usually do not code this class and all the functionally that the Population should have I transfer it to the Genetic Algorithm class. However these are preferences and abstraction level.

On the other hand, the Peasant is the class that will hold information regarding the DNA, mutation function, fitness function... It is also possible to transfer all this functionality to the main Genetic Algorithm class but I would avoid doing so. It is not healthy to give all the resposability to one class, divide and conquer they say...

## Fitness function

This is the function that will calculate an error for each peasant. It can be as complex as your DNA. It we have a DNA that describes the shape of a Neural Network, then, the fitness function will run the Neural Network and obtain a result which will use to get an error. For something more simple (with the example of 0s and 1s) we could describe a fitness function which converst the DNA to integers and sum over them.

DNA = "0010011"

```python
import numpy as np

class IntegerSum (object):

    def __init__ (self):
        pass

    def get_fitness(self, _peasant):
        converted_dna = np.array([int(gen) for gen in _peasant.dna])
        self.__fitness = np.abs(np.sum(converted_dna) - converted_dna.shape[0])

        return self.__fitness
```

In this code, if we call get_fitness function from Peasant with DNA = "0010011" it would return us 3. Let's test it.

In [1]:
import numpy as np
from copy import deepcopy

class IntegerSum (object):

    def __init__ (self):
        pass

    def evaluate_peasant(self, _peasant):
        converted_dna = np.array([int(gen) for gen in _peasant.dna])
        _peasant.fitness = np.abs(np.sum(converted_dna) - converted_dna.shape[0])
        
    
# This is a temporal Peasant
class Peasant (object):
    
    def __init__(self, **kwargs):
        self.__dna = kwargs["dna"]
        self.__fitness = None
        
    @property
    def dna(self):
        return deepcopy(self.__dna) # Don't trust Python
    
    @property
    def fitness(self):
        return self.__fitness
    
    @fitness.setter
    def fitness(self, value):
        self.__fitness = value
    

p_1 = Peasant(dna = "0010011")
fitness_function = IntegerSum()

fitness_function.evaluate_peasant(p_1)

print(f"Fitness of P1 with DNA {p_1.dna} = {p_1.fitness}")

Fitness of P1 with DNA 0010011 = 4


## Crossover

This can be very simple or very complex. The most simple crossover between two Peasants is to cut their DNA and mix them which is called the one_point_crossover. The main function would be the following one

In [12]:
import numpy as np
from copy import deepcopy

class OnePointCrossover(object):
    
    def __init__(self):
        pass
    
    def generate_peasant(self, _p1, _p2):
        assert (len(_p1.dna) == len(_p2.dna)) # Check
        r = np.random.choice(len(_p1.dna))
        dna = p_1.dna[:r] + p_2.dna[r:]
        
        return dna


# This is a temporal Peasant
class Peasant (object):
    
    def __init__(self, **kwargs):
        self.__dna = kwargs["dna"]
        self.__fitness = None
        
    @property
    def dna(self):
        return deepcopy(self.__dna) # Don't trust Python
    
    @property
    def fitness(self):
        return self.__fitness
    
    @fitness.setter
    def fitness(self, value):
        self.__fitness = value
    

p_1 = Peasant(dna = "0010011")
p_2 = Peasant(dna = "1111000")
crossover = OnePointCrossover()

# Begin loop of new peasants
for ii in range(10):
    offspring_dna = crossover.generate_peasant(p_1, p_2)
    p_3 = Peasant(dna = offspring_dna)
    
    print(f"Iter {ii} : DNA {p_3.dna}")

Iter 0 : DNA 0111000
Iter 1 : DNA 1111000
Iter 2 : DNA 0010010
Iter 3 : DNA 0011000
Iter 4 : DNA 0010000
Iter 5 : DNA 0011000
Iter 6 : DNA 0010010
Iter 7 : DNA 0111000
Iter 8 : DNA 0011000
Iter 9 : DNA 0010000


The objective of the crossover it to generate a new DNA from two parents' DNAs. The method or technique to do so can be whatever the programmer decides that is suitable for the algorithm.

## Mutation

It is very self explanatory. The mutator function takes an input DNA and, randomly, performs some kind of mutation in the DNA. This mutation should be somewhat relevant in the possible behaviour of the Peasant but still make it functional. Similar to Machine Learning models, this mutator avoid overfitting of the model. In a Genetic Algorithm, the mutator add some randomness to allow new possibilities. I will show you two examples of mutators

In [65]:
import numpy as np
from copy import deepcopy

class OneGenMutator(object):
    
    def __init__(self):
        pass
    
    def mutate_peasant(self, _peasant):
        r = np.random.choice(len(_peasant.dna))
        dna = _peasant.dna[:r] if r != 0 else ""
        dna += str(np.random.choice([0,1]))
        dna += _peasant.dna[r+1:] if r+1 < len(_peasant.dna) else ""
        _peasant.dna = dna


class TwoGenSwapMutator(object):
    
    def __init__(self):
        pass
    
    def mutate_peasant(self, _peasant):
        r = np.random.choice(len(_peasant.dna), 2, False)
        if r[0] > r[1]:
            r = r[[1,0]]
        
        dna = _peasant.dna[:r[0]]
        dna += _peasant.dna[r[1]]
        dna += _peasant.dna[r[0]+1:r[1]]
        dna += _peasant.dna[r[0]]
        dna += _peasant.dna[r[1]+1:]
            
        _peasant.dna = dna


# This is a temporal Peasant
class Peasant (object):
    
    def __init__(self, **kwargs):
        self.__dna = kwargs["dna"]
        self.__fitness = None
        
    @property
    def dna(self):
        return deepcopy(self.__dna) # Don't trust Python
    
    @dna.setter
    def dna(self, value):
        self.__dna = value
    
    @property
    def fitness(self):
        return self.__fitness
    
    @fitness.setter
    def fitness(self, value):
        self.__fitness = value
    

p_1 = Peasant(dna = "0010011")
one_gen_mut = OneGenMutator()
two_gen_swap_mut = TwoGenSwapMutator()

print("One Gen Mutator\n")
# Begin loop of OneGenMutator
for ii in range(10):
    p_aux = deepcopy(p_1)   # Again, use deepcopy and never trust python
    one_gen_mut.mutate_peasant(p_aux)
    
    print(f"Iter {ii} : DNA {p_aux.dna}")

print("\nTwo Gen Mutator\n")
# Begin loop of TwoGenMutator
for ii in range(10):
    p_aux = deepcopy(p_1)   # Again, use deepcopy and never trust python
    two_gen_swap_mut.mutate_peasant(p_aux)
    
    print(f"Iter {ii} : DNA {p_aux.dna}")

One Gen Mutator

Iter 0 : DNA 0000011
Iter 1 : DNA 0010011
Iter 2 : DNA 0110011
Iter 3 : DNA 0010111
Iter 4 : DNA 0010011
Iter 5 : DNA 0011011
Iter 6 : DNA 0010011
Iter 7 : DNA 1010011
Iter 8 : DNA 0010011
Iter 9 : DNA 0010011

Two Gen Mutator

Iter 0 : DNA 0010011
Iter 1 : DNA 1000011
Iter 2 : DNA 0110010
Iter 3 : DNA 0110001
Iter 4 : DNA 0010101
Iter 5 : DNA 1010010
Iter 6 : DNA 0001011
Iter 7 : DNA 0010011
Iter 8 : DNA 0010011
Iter 9 : DNA 0011001
